In [1]:
%run ./work/lib.py

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, DecimalType, TimestampType, DataType, IntegerType
from pyspark.sql.functions import col, lit, trim, substring, concat, udf, upper, initcap
from datetime import datetime
import os

In [3]:
conf = SparkConf() \
    .setAppName("projeto_pbi") \
    .setSparkHome('./spark/home')

LAKE_HOME = os.getenv("LAKE_HOME", "/spark/home")

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

22/12/17 22:54:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/17 22:54:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
TABLE_NAME = 'fact_order_review'

In [5]:
df = spark.read.csv('{}/dataset/e-commerce/01_extract/order_reviews_dataset.csv'.format(LAKE_HOME), header=True)

df_review = \
    df.select(
        col('review_id').cast(StringType()).alias('ID_REVIEW'),
        col('order_id').cast(StringType()).alias('ID_ORDER'),
        col('review_score').cast(IntegerType()).alias('VAL_SCORE_REVIEW'),
        col('review_creation_date').cast(TimestampType()).alias('DAT_CREATION'),
        col('review_answer_timestamp').cast(TimestampType()).alias('DAT_ANSWER'),
    )\
    .where(col('review_creation_date') != 'null')
    

order_sk = map_sk(spark=spark, table_name='fact_order')

In [6]:
fact_order_review = \
    df_review \
        .withColumn('SK_ORDER', locate_sk(order_sk)(col('ID_ORDER')))

In [7]:
fact_order_review = \
    fact_order_review.select(
            col('ID_REVIEW'),
            col('SK_ORDER'),
            col('VAL_SCORE_REVIEW'),
            col('DAT_CREATION'),
            col('DAT_ANSWER'),  
        )

fact_order_review.write.mode('overwrite').csv('{}/dataset/e-commerce/04_fact/{}.csv'.format(LAKE_HOME, TABLE_NAME), header=True)

In [8]:
spark.stop()